# Detecting moving vechiles in video 

### 1. Read the video: 

In [2]:
from IPython.display import Video

In [4]:
Video('https://drive.google.com/file/d/1HxxhIf4dM7VILIb5-dR3qXDbeGPOLgv-/view')

In [11]:
import urllib.request

In [16]:
urllib.request.urlretrieve('https://drive.google.com/file/d/1HxxhIf4dM7VILIb5-dR3qXDbeGPOLgv-/view', 'video_name1.mp4') 

('video_name1.mp4', <http.client.HTTPMessage at 0x1170a1da0>)

In [17]:
Video('video_name1.mp4')

### 2. Preprocessing: 

In [18]:
import sys
import cv2
import matplotlib.pyplot as plt
import math
import numpy as np

In [19]:
SCALAR_BLACK = (0.0,0.0,0.0)
SCALAR_WHITE = (255.0,255.0,255.0)
SCALAR_YELLOW = (0.0,255.0,255.0)
SCALAR_GREEN = (0.0,255.0,0.0)
SCALAR_RED = (0.0,0.0,255.0)
SCALAR_CYAN = (255.0,255.0,0.0)